In [1]:
import sqlite3
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html, Input, Output, State

In [2]:
# Connect to SQLite database and fetch data
conn = sqlite3.connect('mydatabase.db')
query = """
SELECT m.id, m.age, m.job, m.marital, m.education, m."default", m.balance,
       m.housing, m.loan, m.contact, m.day, m.month, m.duration, m.campaign,
       m.pdays, m.previous, m.y, p.poutcome
FROM main_table m
JOIN p_outcome_table p ON m.id = p.id
"""
full_data = pd.read_sql_query(query, conn)
conn.close()

In [3]:
# Initialize the Dash app
app = dash.Dash(__name__)
app.title = "Enhanced Bank Data Analysis Dashboard"

In [4]:
# Layout of the dashboard
app.layout = html.Div([
    html.H1("Bank Data Analysis Dashboard", style={"textAlign": "center", "font-family": "Arial, sans-serif", "color": "#333"}),
    
    # Filters
    html.Div([
        html.Div([
            html.Label("Select Job Type:"),
            dcc.Dropdown(
                id='job-filter',
                options=[{'label': job, 'value': job} for job in full_data['job'].unique()],
                multi=True,
                placeholder="Select Job Type"
            ),
            html.Label("Select Marital Status:"),
            dcc.Dropdown(
                id='marital-filter',
                options=[{'label': status, 'value': status} for status in full_data['marital'].unique()],
                multi=True,
                placeholder="Select Marital Status"
            ),
            html.Label("Select Education:"),
            dcc.Dropdown(
                id='education-filter',
                options=[{'label': edu, 'value': edu} for edu in full_data['education'].unique()],
                multi=True,
                placeholder="Select Education"
            ),
            html.Label("Select Month:"),
            dcc.Dropdown(
                id='month-filter',
                options=[{'label': month, 'value': month} for month in full_data['month'].unique()],
                multi=True,
                placeholder="Select Month"
            ),
            html.Label("Select Balance Range:"),
            dcc.RangeSlider(
                id='balance-slider',
                min=full_data['balance'].min(),
                max=full_data['balance'].max(),
                step=100,
                marks={int(i): str(i) for i in range(int(full_data['balance'].min()), int(full_data['balance'].max()), 10000)},
                value=[full_data['balance'].min(), full_data['balance'].max()]
            ),
            html.Button('Apply Filters', id='apply-filters', n_clicks=0),
            html.Button('Reset Filters', id='reset-filters', n_clicks=0)
        ], style={'display': 'flex', 'flex-direction': 'column', 'gap': '10px', 'width': '30%', 'margin': '0 auto'}),
    ], style={'background': '#f0f2f5', 'padding': '20px', 'border-radius': '10px'}),

    # Graphs
    html.Div([
        dcc.Graph(id='scatter-plot'),
        dcc.Graph(id='correlation-heatmap'),
        dcc.Graph(id='education-pie-chart'),
        dcc.Graph(id='campaign-bar-chart')
    ], style={'margin-top': '20px', 'padding': '20px', 'background': '#f8f9fa', 'border-radius': '10px', 'box-shadow': '0px 4px 10px rgba(0, 0, 0, 0.1)'})
])

In [5]:
# Callback to update graphs based on user input
@app.callback(
    [Output('scatter-plot', 'figure'),
     Output('correlation-heatmap', 'figure'),
     Output('education-pie-chart', 'figure'),
     Output('campaign-bar-chart', 'figure')],
    [Input('apply-filters', 'n_clicks')],
    [State('job-filter', 'value'),
     State('marital-filter', 'value'),
     State('education-filter', 'value'),
     State('month-filter', 'value'),
     State('balance-slider', 'value')]
)
def update_graphs(n_clicks, selected_jobs, selected_marital, selected_edu, selected_month, selected_balance):
    # Filter the data
    filtered_data = full_data[
        (full_data['job'].isin(selected_jobs if selected_jobs else full_data['job'])) &
        (full_data['marital'].isin(selected_marital if selected_marital else full_data['marital'])) &
        (full_data['education'].isin(selected_edu if selected_edu else full_data['education'])) &
        (full_data['month'].isin(selected_month if selected_month else full_data['month'])) &
        (full_data['balance'] >= selected_balance[0]) & (full_data['balance'] <= selected_balance[1])
    ]

# Scatter Plot: Duration vs Balance
    scatter_fig = px.scatter(
        filtered_data, x='duration', y='balance',
        color='age', title="Duration vs Balance",
        labels={"duration": "Duration", "balance": "Balance"}
    )
    scatter_fig.update_layout(plot_bgcolor="#fff", paper_bgcolor="#f8f9fa", font=dict(family="Arial", size=12))

    # Correlation Heatmap
    numeric_cols = filtered_data.select_dtypes(include=['number']).corr()
    heatmap_fig = px.imshow(
        numeric_cols, text_auto=True, title="Correlation Heatmap",
        color_continuous_scale='Blues'
    )
    heatmap_fig.update_layout(plot_bgcolor="#fff", paper_bgcolor="#f8f9fa", font=dict(family="Arial", size=12))

    # Pie Chart: Education Distribution
    pie_fig = px.pie(
        filtered_data, names='education', title="Education Distribution",
        hole=0.4
    )
    pie_fig.update_traces(textinfo='percent+label')
    pie_fig.update_layout(paper_bgcolor="#f8f9fa")

    # Bar Chart: Campaign vs Success Rate
    bar_fig = px.bar(
        filtered_data, x='campaign', y='y',
        color='y', barmode='group', title="Campaign Success Rate",
        labels={"campaign": "Number of Contacts", "y": "Success (Yes/No)"}
    )
    bar_fig.update_layout(plot_bgcolor="#fff", paper_bgcolor="#f8f9fa", font=dict(family="Arial", size=12))

    return scatter_fig, heatmap_fig, pie_fig, bar_fig

In [ ]:
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)